In [28]:
import os
import scanpy as sc
import scvi
import torch

In [29]:
in_data_dir = "/local/home/savchuki/projects/swarm-atlas/data"
out_data_dir = "/local/home/savchuki/projects/swarm-atlas/data"
in_filename = "lung-atlas-public.h5ad"
out_biopsy_filename = "lung-atlas-public-biopsy-hvgs.h5ad"
out_transplant_filename = "lung-atlas-public-transplant-hvgs.h5ad"
out_biopsy_path = os.path.join(out_data_dir, out_biopsy_filename)
out_transplant_path = os.path.join(out_data_dir, out_transplant_filename)

torch.set_float32_matmul_precision("high")

In [30]:
# Load the raw adata
adata_path = os.path.join(in_data_dir, in_filename)
adata = sc.read(adata_path)
adata

AnnData object with n_obs × n_vars = 32472 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

In [31]:
adata_biopsy = adata[adata.obs['dataset'] == '10x_Biopsy']
adata_biopsy

View of AnnData object with n_obs × n_vars = 10046 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

In [32]:
adata_transplant = adata[(adata.obs['dataset'] == '10x_Transplant') | (adata.obs['dataset'] == 'Dropseq_Transplant')]
adata_transplant

View of AnnData object with n_obs × n_vars = 12725 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

In [33]:
adata_biopsy.raw = adata_biopsy  # keep full dimension safe
sc.pp.highly_variable_genes(
    adata_biopsy,
    flavor="seurat_v3",
    n_top_genes=2000,
    layer="counts",
    batch_key="batch",
    subset=True,
)

adata_transplant.raw = adata_transplant
sc.pp.highly_variable_genes(
    adata_transplant,
    flavor="seurat_v3",
    n_top_genes=2000,
    layer="counts",
    batch_key="batch",
    subset=True,
)

/home/savchuki/miniconda3/envs/swarm-atlas/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:72: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(
/home/savchuki/miniconda3/envs/swarm-atlas/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:72: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


In [34]:
# Write processed adata
adata.write_h5ad(out_transplant_path)
adata.write_h5ad(out_biopsy_path)